## Modelo de optimización - Gestión de activos

Referencias:

An efficient mixed-integer linear formulation for long-term overhead lines maintenance scheduling in power distribution systems,’’ IEEE Trans. Power Del., vol. 24, no. 4, pp. 2043–2053, Oct. 2009]).

### Librerías

In [1]:
import pandas as pd #manejo de datos
from pyomo.environ import * #optimización
import numpy as np

In [2]:
def create_dict(df):
    d = {}
    for i in df.index:
        for j in df.columns:
            d[i,j] = df.loc[i,j]
    return d

def T_dict(T, array):
    data = pd.DataFrame(index = T, data={"data":array})["data"].to_dict()
    return data

### Crear modelo

In [3]:
model = ConcreteModel()

### Sets

In [4]:
# Alimentadores (indice i)
N = list(range(1,12))
model.N = Set(initialize=N)
# Tasas de falla desacopladas o tareas de mantenimiento (indice k)
ND = ["minor","major","tree"]
model.ND = Set(initialize=ND)
# Periodo de planeación
T = list(range(1,11))
model.T = Set(initialize=T)

### Parámetros

In [5]:
# Precio por hora necesaria para mantenimiento preventivo ($/h)
model.C_labor_maintenance =  Param(model.ND, initialize = {"minor":2.5, "major":2.5, "tree":5}) 

# Precio por hora necesaria para reparación por falla ($/h)
model.C_labor_repair=  Param(model.ND, initialize = {"minor":2.5, "major":2.5, "tree":2.5}) 

# Precio de los materiales usados ​​necesarios para la actividad k de mantenimiento del alimentador i ($/km)
C_MMaterial = pd.read_excel("Data.xlsx", "C_MMaterial", index_col=0)
model.C_MMaterial = Param(model.N, model.ND, initialize = create_dict(C_MMaterial), domain = Any)

# Precio de los materiales usados ​​necesarios para la reparación de la averia k ocurrida en el alimentador i ($)
C_RMaterial = pd.read_excel("Data.xlsx", "C_RMaterial", index_col=0)
model.C_RMaterial = Param(model.N, model.ND, initialize = create_dict(C_RMaterial), domain = Any)

# Duración promedio de la interrupción vista por el cliente j debido a la actividad de mantenimiento k del alimentador i (h).
model.d_POutage = Param(model.ND, initialize = {"minor":1, "major":2, "tree":0}) 

# Duración promedio de la interrupción vista por el cliente j debido a una falla k del alimentador i (h).
model.d_UPOutage = Param(model.ND, initialize = {"minor":1, "major":6, "tree":1}) 


# Número de horas de trabajo necesarias para la actividad de mantenimiento k del alimentador i (h/km)
Labor_maintenance = pd.read_excel("Data.xlsx", "Labor_maintenance", index_col=0)
model.Labor_maintenance = Param(model.N, model.ND, initialize = create_dict(Labor_maintenance), domain = Any)

# Número de horas de trabajo necesarias para la reparación k del alimentador i (h)
Labor_repair = pd.read_excel("Data.xlsx", "Labor_repair", index_col=0)
model.Labor_repair = Param(model.N, model.ND, initialize = create_dict(Labor_repair), domain = Any)

feeder_data = pd.read_excel("Data.xlsx", "Feeder", index_col=0)
# Potencia media en cada alimentador i (MW)
model.P = Param(model.N, initialize= T_dict(N, feeder_data["Average load"].to_numpy()))
# Numero de consumidores por alimentador
NC = np.sum(feeder_data["customers"].to_numpy())
model.NC = Param(model.N, initialize= T_dict(N, feeder_data["customers"].to_numpy()))
# Costo de energía no suministrada planeada por megavatio-hora para el alimentador i ($/MWh)
model.C_PUE = Param(model.N, initialize= T_dict(N, feeder_data["C_PUE"].to_numpy()))
# Costo de energía no suministrada no planificada por megavatio-hora para el alimentador i ($/MWh)
model.C_UPUE = Param(model.N, initialize= T_dict(N, feeder_data["C_UPUE"].to_numpy()))
# Precio promedio de venta de energía por megavatio-hora en el alimentador i ($/MWh)
model.PE = Param(model.N, initialize= T_dict(N, feeder_data["PE"].to_numpy()))
# Longitud de cada alimentador en km
model.L = Param(model.N, initialize= T_dict(N, feeder_data["Length"].to_numpy()))

# Número de clientes afectados por el mantenimiento k en el alimentador i
NC_M = pd.read_excel("Data.xlsx", "Customers_maintenance", index_col=0)
model.NC_M = Param(model.N, model.ND, initialize = create_dict(NC_M), domain = Any)

# Número de clientes afectados por la k falla ocurrida en el alimentador i
NC_F = pd.read_excel("Data.xlsx", "Customers_failure", index_col=0)
model.NC_F = Param(model.N, model.ND, initialize = create_dict(NC_F), domain = Any)

# Tasa de incremento anual de la demanda
model.q = Param(initialize = 0.03)
# Tasa de descuento anual
model.DR = Param(initialize = 0.1)


t_data = pd.read_excel("Data.xlsx", "t_data", index_col=0)
# Presupuesto asignado a mantenimiento en el periodo t
model.Budget = Param(model.T, initialize= T_dict(t_data.index, t_data["Budget"].to_numpy()))
# Número de horas de trabajo disponibles en el periodo t para mantenimiento menor y mayor o reparación de averías menores y mayores
model.Labor = Param(model.T, initialize= T_dict(t_data.index, t_data["Labor"].to_numpy()))
# Número de horas de trabajo disponibles en el periodo t para poda de árboles o eliminación de fallas relacionadas con la vegetación.
model.Labor_tt = Param(model.T, initialize= T_dict(t_data.index, t_data["Labor_tt"].to_numpy()))
# SAIFI máximo por año
model.SAIFI = Param(model.T, initialize= T_dict(t_data.index, t_data["SAIFI"].to_numpy()))
# SAIDI máximo por año
model.SAIDI = Param(model.T, initialize= T_dict(t_data.index, t_data["SAIDI"].to_numpy()))


K = pd.read_excel("Data.xlsx", "Failure_rate", index_col=0)
model.K = Param(list(range(0,11)), model.ND, initialize = create_dict(K), domain = Any)

In [6]:
T = list(range(1,6))
model.T = Set(initialize=T)

    'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
    This is usually indicative of a modelling error. To avoid this warning,
    use block.del_component() and block.add_component().


### Variables

In [7]:
# Variable binaria que es igual a 1 si la actividad de mantenimiento k se realiza en el alimentador i en el período t y 0 en caso contrario
model.X = Var(model.ND, model.N, model.T, within=Binary) 
# k tasa de falla desacoplada del alimentador i en el período t que depende de la última vez que se realizó la actividad de mantenimiento en este alimentador (es decir, falla menor, falla mayor y poda de árboles).
model.lam = Var(model.ND, model.N, model.T, within=NonNegativeReals) 

### Funcion objetivo

In [8]:
def obj_rule(m):
    return sum((((m.C_labor_maintenance[k]*m.Labor_maintenance[i,k] + m.C_MMaterial[i,k])*m.L[i] + m.P[i]*m.d_POutage[k]*(m.C_PUE[i]+m.PE[i])*((1+m.q)**(t-1)))*m.X[k,i,t] +
                ((m.C_labor_repair[k]*m.Labor_repair[i,k] + m.C_RMaterial[i,k]) + m.P[i]*m.d_UPOutage[k]*(m.C_UPUE[i]+m.PE[i])*((1+m.q)**(t-1)))*m.lam[k,i,t])*(1/((1+m.DR)**t))
                for k in m.ND for i in m.N for t in m.T)                         

model.Obj=Objective(rule=obj_rule, sense=minimize)                  #Objetive=Objetive, maximizar Valor presente neto

### Restricciones

In [9]:
def budget_rule(m,t):
    return sum(((m.C_labor_maintenance[k]*m.Labor_maintenance[i,k] + m.C_MMaterial[i,k])*m.L[i] + m.P[i]*m.d_POutage[k]*(m.C_PUE[i]+m.PE[i])*((1+m.q)**(t-1)))*m.X[k,i,t] +
            ((m.C_labor_repair[k]*m.Labor_repair[i,k] + m.C_RMaterial[i,k]) + m.P[i]*m.d_UPOutage[k]*(m.C_UPUE[i]+m.PE[i])*((1+m.q)**(t-1)))*m.lam[k,i,t] for k in m.ND for i in m.N) <= m.Budget[t]
model.budget_rule=Constraint(model.T,rule=budget_rule)

def labor_rule(m,t):
    return sum(m.Labor_maintenance[i,k]*m.L[i]*m.X[k,i,t] + m.Labor_repair[i,k]*m.lam[k,i,t] for k in ND[:-1] for i in m.N) <= m.Labor[t]
model.labor_rule=Constraint(model.T,rule=labor_rule)

def labor_tt_rule(m,t):
    return sum(m.Labor_maintenance[i,k]*m.L[i]*m.X[k,i,t] + m.Labor_repair[i,k]*m.lam[k,i,t] for k in [ND[-1]] for i in m.N) <= m.Labor[t]
model.labor_tt_rule=Constraint(model.T,rule=labor_tt_rule)


In [10]:
def saifi_rule(m,t):
    return sum(m.lam[k,i,t]*(m.NC_F[i,k]/NC) + m.X[k,i,t]*(m.NC_M[i,k]/NC) for k in model.ND for i in m.N) <= m.SAIFI[t]
model.saifi_rule = Constraint(model.T,rule=saifi_rule)

def saidi_rule(m,t):
    return sum(m.lam[k,i,t]*(m.d_UPOutage[k]/NC) + m.X[k,i,t]*(m.d_POutage[k]/NC) for k in model.ND for i in m.N) <= m.SAIDI[t]
model.saidi_rule = Constraint(model.T,rule=saidi_rule)

def single_minor_major(m,t,i):
    return sum(m.X[k,i,t] for k in ND[:-1]) <= 1
model.single_minor_major = Constraint(model.T, model.N,rule=single_minor_major)

In [11]:
def falure_rate_din(m,i,t,j,k):
    if t >= j:
        return m.lam[k,i,t] >= m.K[j,k]*m.L[i]*(1-sum(m.X[k,i,t-n] for n in list(range(0,j)))) 
    else:
        return Constraint.Skip
model.falure_rate_din = Constraint(model.N, model.T, model.T, model.ND, rule=falure_rate_din)

#def falure_rate_din1(m,i,t,k):    
#    return m.lam[k,i,t] >= m.K[1,k]*m.L[i]*(1-m.X[k,i,t])     
#model.falure_rate_din1 = Constraint(model.N, model.T, model.ND, rule=falure_rate_din1)

#def falure_rate_din2(m,i,t,k):
#    if t >= 2:
#        return m.lam[k,i,t] >= m.K[2,k]*m.L[i]*(1 - m.X[k,i,t] - m.X[k,i,t-1]) 
#    else:
#        return Constraint.Skip    
#model.falure_rate_din2 = Constraint(model.N, model.T, model.ND, rule=falure_rate_din2)


def falure_rate_0(m,i,t,k):
    return m.lam[k,i,t] >= m.K[0,k]*m.L[i]*m.X[k,i,t]
model.falure_rate_0 = Constraint(model.N, model.T, model.ND, rule=falure_rate_0)

In [12]:
solver = SolverFactory('cplex')
#solver.options['mip tolerances mipgap'] = 1e-2
    
#solver = SolverFactory('gurobi')
#solver.options['mipgap'] = 1e-2

#results = neos.solve(model, opt=solver)
results=solver.solve(model, tee=True, keepfiles=True) 

Solver script file=C:\Users\Nicolas\AppData\Local\Temp\tmpzmcomijp.cplex.script
Solver log file: 'C:\Users\Nicolas\AppData\Local\Temp\tmpg6h74zhe.cplex.log'
Solver solution file: 'C:\Users\Nicolas\AppData\Local\Temp\tmpk7s7yie9.cplex.sol'
Solver problem files: ('C:\\Users\\Nicolas\\AppData\\Local\\Temp\\tmp2_h9hi1a.pyomo.lp',)

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Nicolas\AppData\Local\Temp\tmpg6h74zhe.cplex.log' open.
CPLEX> Problem 'C:\Users\Nicolas\AppData\Local\Temp\tmp2_h9hi1a.pyomo.lp' read.
Read time = 0.00 sec. (0.08 ticks)
CPLEX> Problem name         : C:\Users\Nicolas\AppData\Local\Temp\tmp2_h9hi1a.pyomo.lp
Obje

In [13]:
model.X.display()

X : Size=165, Index=X_index
    Key              : Lower : Value                   : Upper : Fixed : Stale : Domain
     ('major', 1, 1) :     0 :                     0.0 :     1 : False : False : Binary
     ('major', 1, 2) :     0 :                     0.0 :     1 : False : False : Binary
     ('major', 1, 3) :     0 :                     0.0 :     1 : False : False : Binary
     ('major', 1, 4) :     0 :                    -0.0 :     1 : False : False : Binary
     ('major', 1, 5) :     0 :                    -0.0 :     1 : False : False : Binary
     ('major', 2, 1) :     0 :                    -0.0 :     1 : False : False : Binary
     ('major', 2, 2) :     0 :                    -0.0 :     1 : False : False : Binary
     ('major', 2, 3) :     0 :                    -0.0 :     1 : False : False : Binary
     ('major', 2, 4) :     0 :                    -0.0 :     1 : False : False : Binary
     ('major', 2, 5) :     0 :                    -0.0 :     1 : False : False : Binary
    

In [14]:
model.Obj.display()

Obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 351565.23396493576
